# Stocks and Data collection

### The stocks selected for this project from the indian market are:

- Tata Steel Limited - TATASTEEL.NS 
- Federal Bank Limited - FEDERALBNK.NS
- Jubilant FoodWorks Limited - JUBLFOOD.NS
- Dr. Reddy's Laboratories Limited - DRREDDY.NS
- GAIL (India) Limited - GAIL.NS

### The libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import scipy.optimize as sco
import matplotlib.cm as cm
import datetime as dt
import plotly.graph_objects as go

### Stock Price Data Collection

In [6]:
tickers = ['TATASTEEL.NS','FEDERALBNK.NS','JUBLFOOD.NS','DRREDDY.NS','GAIL.NS']
stock_data = pd.DataFrame()
years = 5 # 5 year period is taken for this analysis
e_date = dt.datetime.now()
s_date = e_date - dt.timedelta(days=365 * years)

for ticker in tickers:
    data = yf.download(ticker, start=s_date, end=e_date)
    stock_data[ticker] = data['Close']
stock_data # Closing Prices for Past 5 yrs is stored in this Data Frame

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,TATASTEEL.NS,FEDERALBNK.NS,JUBLFOOD.NS,DRREDDY.NS,GAIL.NS
Date,,,,,
2020-08-12,35.683376,51.929424,369.125366,875.669189,52.505764
2020-08-13,35.562851,51.929424,374.857513,872.587585,52.157688
2020-08-14,36.027718,50.583107,370.194550,875.911377,51.381218
2020-08-17,36.591599,50.823517,373.679413,875.863037,52.291561
2020-08-18,37.405132,51.448597,379.322449,876.105164,51.916710
...,...,...,...,...,...
2025-08-05,159.619995,196.039993,641.500000,1214.800049,171.339996
2025-08-06,158.660004,197.369995,630.049988,1196.699951,169.910004
2025-08-07,159.669998,198.630005,635.700012,1200.800049,169.490005


## Plotting the Prices of these stocks over time

In [8]:
fig1 = go.Figure()
for ticker in stock_data.columns:
    fig1.add_trace(go.Scatter(x=stock_data.index, y=stock_data[ticker], mode='lines', name=ticker))

fig1.update_layout(
    title='Stock Price Trends',
    xaxis_title='Date',
    yaxis_title='Close Price',
    legend_title='Ticker'
)
fig1.show()

### Calculating Returns
Here we take Log returns as They are additive over time, exhibit symmetry between gains and losses, and tend to be more normally distributed, making them suitable for financial modeling and analysis.

In [9]:
log_returns = np.log(stock_data/stock_data.shift(1))
log_returns = log_returns.dropna()
log_returns

,TATASTEEL.NS,FEDERALBNK.NS,JUBLFOOD.NS,DRREDDY.NS,GAIL.NS
Date,,,,,
2020-08-13,-0.003383,0.000000,0.015410,-0.003525,-0.006651
2020-08-14,0.012987,-0.026268,-0.012517,0.003802,-0.014999
2020-08-17,0.015530,0.004742,0.009370,-0.000055,0.017562
2020-08-18,0.021989,0.012224,0.014988,0.000276,-0.007194
2020-08-19,-0.006465,0.025832,0.013558,-0.006915,0.045371
...,...,...,...,...,...
2025-08-05,0.000376,-0.002140,-0.011315,-0.008361,-0.018504
2025-08-06,-0.006032,0.006761,-0.018010,-0.015012,-0.008381
2025-08-07,0.006346,0.006364,0.008928,0.003420,-0.002475


## Plotting the Prices of these stocks over time

In [17]:
corr_matrix = log_returns.corr()

#Heat_Map to show the correlation
fig_corr = go.Figure(
    data=go.Heatmap(
        z=corr_matrix.values,
        x=corr_matrix.columns,
        y=corr_matrix.columns,
        colorscale='RdYlGn',
        zmin=-1,
        zmax=1,
        colorbar_title='Correlation'
    )
)

fig_corr.update_layout(
    title='Correlation Heatmap of Log Returns',
    xaxis_title='Ticker',
    yaxis_title='Ticker'
)

fig_corr.show()

Here we see the only significant positive correlation we see is between GAIL and TATA STEEL

## Monte Carlo Approach

The Monte Carlo approach is a computational technique that uses random sampling to model and analyze systems with uncertainty. It involves running a large number of simulations, each with randomly generated inputs, to estimate the range and likelihood of possible outcomes.

### Monte Carlo Simulation 
It simulates 20,000 random portfolios by assigning random weights to each stock, calculates their annualized return, volatility, and Sharpe ratio, and stores the results. From these simulations, it identifies and outputs the portfolio with the maximum Sharpe ratio (best risk-adjusted return) and the portfolio with the minimum volatility (lowest risk).

In [21]:
num_assets = len(log_returns.columns)
num_simulation = 20000
risk_free_rate = 6.66 #We use this for the purpose of this project

daily_returns = log_returns

# Annualize the returns (assuming 252 trading days in a year)
annual_returns = (daily_returns.mean() * 252) + 1

# Calculate the covariance matrix of the log returns
cov_daily = daily_returns.cov()
cov_annual = cov_daily * 252

# Set a seed for reproducibility
np.random.seed(3)

# Initialize an empty DataFrame to store portfolio results
columns = ['Return', 'Volatility', 'Sharpe Ratio'] + [symbol + ' Weight' for symbol in log_returns.columns]
portfolio_df = pd.DataFrame(columns=columns)

# Run the Monte Carlo simulation for portfolio optimization
for i in range(num_simulation):
    weights = np.random.random(num_assets)
    weights /= np.sum(weights)
    
    portfolio_return = np.dot(weights, annual_returns)
    portfolio_volatility_ = np.sqrt(np.dot(weights.T, np.dot(cov_annual, weights)))
    sharpe_ratio_ = (portfolio_return - risk_free_rate / 252) / portfolio_volatility_

    # Store the results in the DataFrame
    portfolio_df.loc[i] = [portfolio_return - 1, portfolio_volatility_, sharpe_ratio_] + list(weights)

# Find the portfolio with the maximum Sharpe ratio
max_sharpe_portfolio = portfolio_df.loc[portfolio_df['Sharpe Ratio'].idxmax()]

# Find the portfolio with the minimum volatility
min_volatility_portfolio = portfolio_df.loc[portfolio_df['Volatility'].idxmin()]

# Display the portfolios with max Sharpe ratio and min volatility
print(f"\nMaximum Sharpe Ratio Portfolio:")
print(f"Return: {max_sharpe_portfolio['Return']*100:.2f}%")
print(f"Volatility: {max_sharpe_portfolio['Volatility']*100:.2f}%")
print(f"Sharpe Ratio: {max_sharpe_portfolio['Sharpe Ratio']:.2f}\n")
print("WEIGHTS:")
for symbol, weight in zip(log_returns.columns, max_sharpe_portfolio[3:]):
    print(f"{symbol} Weight: {weight*100:.2f}%")

print("-------------------------------------")    

print(f"\nMinimum Volatility Portfolio:")
print(f"Return: {min_volatility_portfolio['Return']*100:.2f}%")
print(f"Volatility: {min_volatility_portfolio['Volatility']*100:.2f}%")
print(f"Sharpe Ratio: {min_volatility_portfolio['Sharpe Ratio']:.2f}\n")
print("WEIGHTS:")
for symbol, weight in zip(log_returns.columns, min_volatility_portfolio[3:]):
    print(f"{symbol} Weight: {weight*100:.2f}%")


Maximum Sharpe Ratio Portfolio:
Return: 15.39%
Volatility: 17.80%
Sharpe Ratio: 6.34

WEIGHTS:
TATASTEEL.NS Weight: 8.53%
FEDERALBNK.NS Weight: 20.15%
JUBLFOOD.NS Weight: 12.64%
DRREDDY.NS Weight: 47.21%
GAIL.NS Weight: 11.46%
-------------------------------------

Minimum Volatility Portfolio:
Return: 14.42%
Volatility: 17.72%
Sharpe Ratio: 6.31

WEIGHTS:
TATASTEEL.NS Weight: 5.39%
FEDERALBNK.NS Weight: 17.23%
JUBLFOOD.NS Weight: 14.36%
DRREDDY.NS Weight: 49.92%
GAIL.NS Weight: 13.11%


## Efficient Frontier Plot (Risk v Return)

The Efficient Frontier plot shows the set of optimal portfolios that deliver the highest possible return for each level of risk. In this chart, risk (volatility) is on the x-axis and expected return is on the y-axis, helping visualize the trade-off between risk and reward.

In [42]:
# Scatter Plot for all simulated portfolios
fig = go.Figure(
    data=go.Scatter(
        x=portfolio_df['Volatility'],
        y=portfolio_df['Return'],
        mode='markers',
        marker=dict(
            color=portfolio_df['Sharpe Ratio'],
            colorscale='RdYlGn',
            size=6,
            line=dict(width=0.5, color='black'),
            colorbar=dict(title='Sharpe Ratio')
        ),
        name='Portfolios'
    )
)

# Marker for Max Sharpe Portfolio
fig.add_trace(go.Scatter(
    x=[max_sharpe_portfolio['Volatility']],
    y=[max_sharpe_portfolio['Return']],
    mode='markers',
    marker=dict(size=14, color='blue', symbol='star'),
    name='Max Sharpe Portfolio'
))

# Marker for Min Volatility Portfolio
fig.add_trace(go.Scatter(
    x=[min_volatility_portfolio['Volatility']],
    y=[min_volatility_portfolio['Return']],
    mode='markers',
    marker=dict(size=14, color='magenta', symbol='star'),
    name='Min Volatility Portfolio'
))

# Layout
fig.update_layout(
    title='Monte Carlo Simulation: Risk vs Return',
    xaxis_title='Portfolio Volatility (Risk)',
    yaxis_title='Portfolio Return',
    legend=dict(x=0.01, y=0.99)
)

fig.show()


From the Efficient Frontier plot, we can infer which portfolios are efficient—those lying on the curve—offering the best return for a given level of risk. Portfolios below the curve are suboptimal, as higher returns could be achieved without taking on more risk, while points above the curve are unattainable given the assets considered.

## Investment Return Calculator

Lets say for an Investment amount of 1L for both the max sharpe and min volatility what are the total risk and return

In [43]:
investment_amount = 100000

# max Sharpe ratio portfolio
expected_annual_return = max_sharpe_portfolio.iloc[0]
expected_volatility = max_sharpe_portfolio.iloc[1]


expected_profit = investment_amount * expected_annual_return
risk_amount = investment_amount * expected_volatility
print("Max Sharpe Ratio portfolio")
print(f"Investment Amount: ₹{investment_amount:,.2f}")
print(f"Expected Annual Return: {expected_annual_return*100:.2f}%")
print(f"Expected Profit: ₹{expected_profit:,.2f}")
print(f"Expected Volatility (Risk): {expected_volatility*100:.2f}%")
print(f"Risk Amount: ₹{risk_amount:,.2f}\n")
print("----------------------------")

# Min Volatility portfolio
expected_annual_return1 = min_volatility_portfolio.iloc[0]
expected_volatility1 = min_volatility_portfolio.iloc[1]


expected_profit1 = investment_amount * expected_annual_return1
risk_amount1 = investment_amount * expected_volatility1
print("Min Volatility portfolio")
print(f"Investment Amount: ₹{investment_amount:,.2f}")
print(f"Expected Annual Return: {expected_annual_return1*100:.2f}%")
print(f"Expected Profit: ₹{expected_profit1:,.2f}")
print(f"Expected Volatility (Risk): {expected_volatility1*100:.2f}%")
print(f"Risk Amount: ₹{risk_amount1:,.2f}\n")


Max Sharpe Ratio portfolio
Investment Amount: ₹100,000.00
Expected Annual Return: 15.39%
Expected Profit: ₹15,394.63
Expected Volatility (Risk): 17.80%
Risk Amount: ₹17,795.84

----------------------------
Min Volatility portfolio
Investment Amount: ₹100,000.00
Expected Annual Return: 14.42%
Expected Profit: ₹14,418.18
Expected Volatility (Risk): 17.72%
Risk Amount: ₹17,723.66

